In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import tempfile

def data_to_speck(data, lon_att, lat_att, alt_att=None, frame=None, alt_unit=None,outfile='test'):

    # TODO: add support for different units, e.g. hour angle

    lon = data[lon_att]
    lat = data[lat_att]

    if alt_att is None:

        # Get cartesian coordinates on unit galactic sphere
        coord = SkyCoord(lon, lat, unit='deg',
                         frame=frame.lower())
        x, y, z = coord.galactic.cartesian.xyz

        # Convert to be on a sphere of radius 100pc
        D = 100
        x *= D
        y *= D
        z *= D

    else:

        distance = data[alt_att]

        # Get cartesian coordinates on unit galactic sphere
        coord = SkyCoord(lon * u.deg, lat * u.deg,
                         distance=distance * u.Unit(alt_unit),
                         frame=frame.lower())
        x, y, z = coord.galactic.cartesian.xyz

        x = x.to_value(u.pc)
        y = y.to_value(u.pc)
        z = z.to_value(u.pc)

    # Create speck table

    tmpfile = './job_talk_data/{}.speck'.format(outfile)

    with open(tmpfile, 'w') as f:

        f.write('datavar 0 colorb_v\n')
        f.write('datavar 1 lum\n')
        f.write('datavar 2 absmag\n')
        f.write('datavar 3 appmag\n')

        for i in range(len(x)):
            f.write('  {0:10.5f} {1:10.5f} {2:10.5f} {3:10.5f} {4:10.5f} {5:10.5f} {6:10.5f}\n'.format(x[i], y[i], z[i], 0., 100., 0., 0.))

    return tmpfile

In [2]:
from astropy.table import Table,vstack

data = Table.read("./job_talk_data/cloud_spines_compiled.fits")

speckfile = data_to_speck(data,'l','b',alt_att='d',frame='galactic',alt_unit='pc',outfile='CloudSpines')